In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [78]:
df = pd.read_csv('Data\Airbnb_Data.csv')
df = pd.DataFrame(df)

In [36]:
df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,...,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,...,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,...,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,...,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [96]:
for i in df["zipcode"]:
    if i != '' :
        i=df["zipcode"].mode()
    else:
        print("No missing values")

In [102]:


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy='most_frequent')



# df['zipcode'] = pd.to_numeric(df['zipcode'], errors='coerce')


df["zipcode"] = imputer.fit_transform(df["zipcode"])
df['zipcode'] = df['zipcode'].astype(int)
df["zipcode"] = df["zipcode"].values.reshape(-1, 1)
std = StandardScaler()

df['zipcode'] = std.fit_transform(df['zipcode'])


ValueError: Expected 2D array, got 1D array instead:
array=[nan nan nan ... nan nan nan].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [60]:
categorical_col = ["property_type","room_type","amenities","bed_type","cancellation_policy","cleaning_fee","city","host_has_profile_pic",
                   "host_identity_verified","instant_bookable","neighbourhood"]

scaling_req_col = ["id", "zipcode"]


for i in scaling_req_col:
    print(df[i].isnull().sum())

0
966


In [50]:
# df['amenities'] = df['amenities'].str.replace('{}', '')
# df['amenities'].split(",")

df['amenities'] = df['amenities'].str.replace('{', '')
df['amenities'] = df['amenities'].str.replace('}', '')


# Split the amenities
sep_col = df['amenities'].str.get_dummies(sep=',')

#Join the amenities DataFrame with the original DataFrame
df = pd.concat([df, sep_col], axis=1)

df = df.drop(['amenities'], axis=1)

In [51]:
df.head()


,id,log_price,property_type,room_type,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,...,Other,Oven,Pool,Refrigerator,Shampoo,Smartlock,Stove,TV,Washer,Waterfront
0,6901257,5.010635,Apartment,Entire home/apt,3,1.0,Real Bed,strict,True,NYC,...,0,0,0,0,0,0,0,0,0,0
1,6304928,5.129899,Apartment,Entire home/apt,7,1.0,Real Bed,strict,True,NYC,...,0,0,0,0,1,0,0,0,1,0
2,7919400,4.976734,Apartment,Entire home/apt,5,1.0,Real Bed,moderate,True,NYC,...,0,0,0,0,1,0,0,1,0,0
3,13418779,6.620073,House,Entire home/apt,4,1.0,Real Bed,flexible,True,SF,...,0,0,0,0,0,0,0,1,1,0
4,3808709,4.744932,Apartment,Entire home/apt,2,1.0,Real Bed,moderate,True,DC,...,0,0,0,0,1,0,0,1,0,0


In [4]:
df["property_type","room_type","amenities","bed_type","cancellation_policy","cleaning_fee","city","host_has_profile_pic",
                   "host_identity_verified","instant_bookable","neighbourhood"].unique()

KeyError: ('property_type', 'room_type', 'amenities', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable', 'neighbourhood')

In [9]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from new_input_features.Columns import *
Data = df.iloc[:10000, :]


Data['host_since'] = pd.to_datetime(Data['host_since'])
Data['host_since_year'] = Data['host_since'].dt.year
Data['host_since_month'] = Data['host_since'].dt.month
Data['host_since_day'] = Data['host_since'].dt.day
Data = Data.drop('host_since', axis=1)
    
# Not using this function on other columns because, the first these columns has way too many missing values


# Data[mean_col] = Data[mean_col].astype(int)

# df['host_response_rate'] = df['host_response_rate'].str.replace('%', '').astype(float)
# df['host_response_rate'] = df['host_response_rate'].astype(int)

# Data["bathrooms","bedrooms","beds"] = Data["bathrooms","bedrooms","beds"].astype(int)

Data.drop(df[good_to_drop_col], axis=1)


imputer = SimpleImputer(strategy='most_frequent')
Data[most_freq_col] = imputer.fit_transform(Data[most_freq_col])

imputer1 = SimpleImputer(strategy='mean')
Data[mean_col] = imputer1.fit_transform(Data[mean_col])

imputer2 = SimpleImputer(strategy='median')
Data[median_col] = imputer2.fit_transform(Data[median_col])


scaler = StandardScaler()
Data[scaling_req_col] = scaler.fit_transform(Data[scaling_req_col])


ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
data_to_encode = Data[scaling_req_col]
encoded_data = ohe.fit_transform(scaling_req_col)
    

encoded_data.shape()

C:\Users\DELL\AppData\Local\Temp\ipykernel_8364\3466710358.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data['host_since'] = pd.to_datetime(Data['host_since'])
C:\Users\DELL\AppData\Local\Temp\ipykernel_8364\3466710358.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data['host_since_year'] = Data['host_since'].dt.year
C:\Users\DELL\AppData\Local\Temp\ipykernel_8364\3466710358.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

ValueError: could not convert string to float: ''

In [ ]:


Data.head()

In [ ]:
df['first_review'] = pd.to_datetime(df['first_review'])
 
df['first_review_year'] = (df['first_review'].dt.year)
df['first_review_month'] = df['first_review'].dt.month
df['first_review_day'] = df['first_review'].dt.day
df = df.drop('first_review', axis=1)

In [ ]:
df.head(30)

In [ ]:
df.isnull().sum()


In [ ]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='median')
np.integer(df['first_review_year','first_review_month', 'first_review_day'])
dl = impute.fit_transform(df['first_review_year', 'first_review_month', 'first_review_day'])

dl = pd.DataFrame(dl)

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df['name'].nunique()